# AI Powered Multi-Agent Deal Finder :

### This notebook creates a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [02:01<00:00, 24.22s/it]


In [4]:
len(deals)

50

In [7]:
deals[0].describe()

"Title: Best Buy Early Black Friday Deals: Up to 60% off + free shipping\nDetails: Best Buy has already kicked off its Early Black Friday sale, with deals on TVs, laptops, Apple devices, and more that are marked down by at least 50%. New doorbusters will go live each week and be added to the sale, so make sure to check back each day. Shipping is free for My Best Buy members (it's free to join). Shop Now at Best Buy\nFeatures: \nURL: https://www.dealnews.com/Best-Buy-Early-Black-Friday-Deals-Up-to-60-off-free-shipping/21781831.html?iref=rss-c142"

In [8]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [9]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [11]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Best Buy Early Black Friday Deals: Up to 60% off + free shipping
Details: Best Buy has already kicked off its Early Black Friday sale, with deals on TVs, laptops, Apple devices, and more that are marked down by at least 50%. New doorbusters will go live each week and be added to the sale, so make sure to check back each day. Shipping is free for My Best Buy members 

In [10]:
def get_recommendations(): 
    # For structured outputs, use openai.beta.chat.completions.parse
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection # Pass the pydantic class here that defines the expected structure
    )
    result = completion.choices[0].message.parsed # not .content, but .parsed
    return result

In [12]:
result = get_recommendations()

In [13]:
len(result.deals)

5

In [14]:
result.deals[1]

Deal(product_description='Experience stunning visuals with the Philips 55" 4K QLED Roku Smart TV. It offers 4K resolution and supports both Dolby Vision and HDR10 for vibrant color and clarity. The integrated Roku interface provides seamless access to your favorite streaming services. This TV combines advanced display technology with user-friendly smart capabilities, making it a perfect entertainment hub for any home.', price=260.0, url='https://www.dealnews.com/products/Philips/Philips-55-4-K-QLED-Roku-Smart-TV/459156.html?iref=rss-c142')

In [15]:
from agents.scanner_agent import ScannerAgent

In [16]:
agent = ScannerAgent()
result = agent.scan()

In [17]:
result

DealSelection(deals=[Deal(product_description='The Refurbished Samsung Galaxy Watch 7 is a stylish and functional smartwatch, featuring a 44mm size, original packaging, and factory settings restored. This model includes all original accessories, ensuring you have everything you need. With a one-year warranty from Allstate, this watch combines modern technology and classic design, allowing you to stay connected and monitor your health seamlessly.', price=106.0, url='https://www.dealnews.com/products/Samsung/Samsung-Galaxy-Watch-7-44-mm-Smartwatch/495284.html?iref=rss-c142'), Deal(product_description='Experience exceptional visual quality with the Philips 55" 4K QLED Roku Smart TV. This television supports 4K resolution, Dolby Vision, and HDR10 capabilities, providing stunning picture quality for your favorite shows and movies. The built-in Roku platform allows access to a wide array of streaming channels, making it easy to enjoy your entertainment seamlessly. It\'s a significant additio